In [4]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
from datetime import datetime, time
import plotly.graph_objects as go
import plotly.express as px

# Load GPS data
gps_data = pd.read_csv('MC2/gps.csv', parse_dates=['Timestamp'])

# Load car assignment data
car_assignment = pd.read_csv('MC2/car-assignments.csv')

# Load the merged transaction data
transaction_data = pd.read_csv('MC2/merged_data.csv', parse_dates=['timestamp'])

# Add a 'FullName' column to car assignment for easier reference
car_assignment['FullName'] = car_assignment['FirstName'] + ' ' + car_assignment['LastName']

# Merge GPS data with car assignment data
merged_data = pd.merge(gps_data, car_assignment, left_on='id', right_on='CarID', how='left')

# Create date picker widget
date_picker = widgets.DatePicker(
    description='Select Date',
    disabled=False,
    value=datetime.strptime('2014-01-06', '%Y-%m-%d')
)

# Create start and end time picker widgets
start_time_picker = widgets.TimePicker(
    description='Start Time',
    value=time(0, 0),
    step=60
)

end_time_picker = widgets.TimePicker(
    description='End Time',
    value=time(23, 59),
    step=60
)

# Function to filter and plot merged GPS data and transaction data
def update_plot(date, start_time, end_time):
    start_datetime = datetime.combine(date, start_time)
    end_datetime = datetime.combine(date, end_time)
    
    # Filter merged data based on the selected date and time range
    filtered_data = merged_data[
        (merged_data['Timestamp'] >= start_datetime) & 
        (merged_data['Timestamp'] <= end_datetime)
    ]
    
    # Filter transaction data based on the selected date and time range
    filtered_transactions = transaction_data[
        (transaction_data['timestamp'] >= start_datetime) & 
        (transaction_data['timestamp'] <= end_datetime)
    ]
    
    # Clear the existing data in the figure
    fig = go.Figure()

    # Add new data to the figure
    for name, group in filtered_data.groupby('FullName'):
        fig.add_trace(go.Scatter(
            x=group['long'],
            y=group['lat'],
            mode='markers+lines',
            name=name,
            text=[f"{name}<br>{row['CurrentEmploymentType']}<br>{row['CurrentEmploymentTitle']}<br>{row['Timestamp']}" for i, row in group.iterrows()],
            hoverinfo='text'
        ))
    
    # Add transactions to the plot
    fig.add_trace(
        go.Scatter(
            x=[None] * len(filtered_transactions),  # Replace with actual longitudes if available
            y=[None] * len(filtered_transactions),  # Replace with actual latitudes if available
            mode='markers',
            marker=dict(color='blue', size=8, symbol='x'),
            name='Transactions',
            text=filtered_transactions.apply(lambda row: f"Transaction at: {row['timestamp']}<br>Location: {row['location']}<br>Amount: {row['price']}<br>CC: {row['cc_last4ccnum']}<br>Loyalty: {row['loyalty_loyaltynum']}", axis=1),
            hoverinfo='text'
        )
    )

    fig.update_layout(
        xaxis_title="Longitude",
        yaxis_title="Latitude",
        width=800,
        height=600
    )

    fig.show()

# Interactive plot
interact(update_plot, date=date_picker, start_time=start_time_picker, end_time=end_time_picker)


FileNotFoundError: [Errno 2] No such file or directory: 'MC2/merged_data.csv'